# 组合权重法
## 算法简介
前面有介绍过几种赋权的方法，有客观的赋权方法如层次分析法，也有客观观的赋权方法如熵权法、变异系数法等。此篇要介绍一种能把他们都组合起来的赋权方法。

客观赋权的方法主要有熵值法、主成分分析法、均方差法、变异系数法( 标准差系数法) 和 CＲITIC 法等其中熵值法主要表现指标数据的离散性，均方差和变异系数法( 标准差系数法) 则主要是利用指标数据的对比强度来刻画指标的重要性，而主成分分析法主要考虑了数据之间的相关性，CRITIC 法则既兼顾到指标数据的相关性( 冲突性) 又考虑到数据之间的对比强度。[1]所以在此可以用CRITIC 法组合权重

## 案例分析
还是用一篇高引用论文为例, 解决一个在市场上选择机器人的多属性决策问题。一个客户要选择机器人，有四个可选择的方案，每个方案有四个指标，分别是：P1表示价格，P2表示速度，P3表示可重复性，P4表示负载能力。针对这四个指标对每一个方案进行综合评价得到最优方案。数据表格如下：
![image-20220130181612356](https://cdn.jsdelivr.net/gh/GEAMING-CHN/images/blogimg/%E6%9D%82%E9%A1%B9/image-20220130181612356.png)

## 原理分析
1. 数据标准化
   
   此篇和之前的CRITIC法一样把正向化和标准化结合
   设有$m$个待评对象，$n$评价指标，可以构成数据矩阵$X=(x_{ij})_{m\times n}$，设标准化后的数据矩阵元素为$x_{ij}^{\prime}$
   若$x_j$为负向指标（越小越优型指标），$P1$、$P3$属于此类指标。
   $$x_{ij}^{\prime}=\frac{\mathrm{max}(x_j)-x_{ij}}{\mathrm{max}(x_j)-\mathrm{min}(x_j)}$$
   若$x_j$为正向指标（越大越优型指标），$P2$、$P4$属于此类指标。
   $$x_{ij}^{\prime}=\frac{x_{ij}-\mathrm{min}(x_j)}{\mathrm{max}(x_j)-\mathrm{min}(x_j)}$$
   
2. 一致性检验

   得到指标权重的方法有很多种、熵权法、变异洗漱法等，再次不进行介绍。设现有三个专家用层次分析法分别得到权重$W_1$，$W_2$，$W_3$，
   $$W_1=(0.3,0.4,0.15,0.15)^T$$
   $$W_2=(0.4,0.3,0.15,0.15)^T$$
   $$W_1=(0.25,0.25,0.25,0.25)^T$$
   同时使用熵权法得到权重$W_4$
   $$W_4=(0.2403,0.2294,0.3062,0.2242)^T$$
   利用Kendall后person一致性系数进行检验。假设有k个变量$W_1,W_2,\cdots,W_k$每个变量有$n$个观测值，即每个变量都是$n$维的。很显然再次我们要对四个权重进行检验，检验通过说明权重具有协同性，检验不通过说明不具有协同性，说明权重间的差异比较大。

3. 组合权重
   
   如检验通过

   设总的权重为$W_c$如果一致性检验通过，则说明各个方法计算出的权重差异不打，我们可以用以下哦公式来计算总权重
   $$W_c=(W_1+W_2+\cdots+W_k)/l$$

   如果检验不通过说明各个方法计算出的权重差异比较大，使用CRITIC法对其求权重，$r_{ij}$表示权重$i$与$j$之间的相关系数，$\sigma_j$代表权重$j$的标准差
   $$C_j=\sigma_j\sum_{i=1}^{m}(1-r_{ij})$$
   $$\theta_j=\frac{C_j}{\sum_{j=1}^nC_j}$$
   $$W_c=\theta_1W_1+\theta_2W_2+\cdots+\theta_kW_k$$

4. 得到评分
   
   $$S=\sum_n^{j=1}Wx_{ij}^{\prime}$$

In [6]:
import copy
import pandas as pd
import numpy as np
import scipy.stats as stats

In [7]:
data=pd.read_excel('机器人方案.xlsx')
data

,方案,P1,P2,P3,P4
0,方案1,3.0,1.0,1.0,70
1,方案2,2.5,0.8,0.8,50
2,方案3,1.8,0.5,2.0,110
3,方案4,2.2,0.7,1.2,90


In [8]:
# 数据正向化处理
label_need = data.keys()[1:]
data1 = data[label_need].values
data2 = copy.deepcopy(data1)
[m, n] = data2.shape
index_all = np.arange(n)
index = [0, 2]  # 负向指标位置,注意python是从0开始计数，对应位置也要相应减1
for j in index:
    d_max = max(data1[:, j])
    d_min = min(data1[:, j])
    data2[:, j] = (d_max - data1[:, j]) / (d_max - d_min)
# 正向指标位置
index = np.delete(index_all, index)
for j in index:
    d_max = max(data1[:, j])
    d_min = min(data1[:, j])
    data2[:, j] = (data1[:, j] - d_min) / (d_max - d_min)
data2


array([[0.        , 1.        , 0.83333333, 0.33333333],
       [0.41666667, 0.6       , 1.        , 0.        ],
       [1.        , 0.        , 0.        , 1.        ],
       [0.66666667, 0.4       , 0.66666667, 0.66666667]])

In [17]:
#一致性检验
#各个赋权方法得到的权重
W1=[0.3,0.4,0.15,0.15]
W2=[0.4,0.3,0.15,0.15]
W3=[0.25,0.25,0.25,0.25]
W4=[0.2403,0.2294,0.3062,0.2242]
W=[W1,W2,W3,W4]
r=np.corrcoef(W)   #求皮尔逊相关系数
r=np.nan_to_num(r) #将nan的值转化为0
the=np.std(W,axis=1)
r1=abs(r)
t2=np.sqrt((len(r)-2) / (1-r1**2))
t2[np.isinf(t2)]=0
T=r1*t2 #p-values 矩阵
alpha = 0.05 #显著性检验
t_score = stats.t.isf(alpha/2, df = len(r)-2)
if np.sum(T)>t_score*len(r)*len(r): 
    print('一致性检验不通过')
    the=np.std(W,axis=1)
    f=np.sum(1-r,axis=1)
    c=the*f
    w=c/sum(c)  #计算权重
    W=np.array(W)
    Wc1=copy.deepcopy(W*w)
    for i in range(0,len(w)):
        Wc1[i,:]=W[i,:]*w[i]
    Wc=np.sum(Wc1,axis=0) 
else:
    print('一致性检验通过')
    Wc=np.sum(W,axis=0)
    Wc=Wc/sum(Wc)

一致性检验不通过


<ipython-input-17-aadeb1e6f633>:12: RuntimeWarning: divide by zero encountered in true_divide
  t2=np.sqrt((len(r)-2) / (1-r1**2))


In [18]:
#计算得分
s=np.dot(data2,Wc)
Score=100*s/max(s) #计算得分
for i in range(0,len(Score)):
    print(f"{data['方案'][i]}百分制评分为：{Score[i]}") 

方案1百分制评分为：91.85091826552814
方案2百分制评分为：88.57894552940891
方案3百分制评分为：85.10401719903089
方案4百分制评分为：100.0
